# Lab-08-1 Perceptron

In [1]:
import torch
import torch.nn.functional as F

## Single Layer Perceptron

#### Can XOR be solved with a single perceptron?

In [2]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = torch.FloatTensor([[0], [1], [1], [0]])

In [3]:
linear = torch.nn.Linear(X.size(1), Y.size(1), bias=True)
sigmoid = torch.nn.Sigmoid()
model = torch.nn.Sequential(linear, sigmoid)

In [4]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [5]:
n_epoch = 10**4
for epoch in range(n_epoch):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, Y)
    loss.backward()
    optimizer.step()
    
    if epoch % 1000 == 0:
        print('Epoch: {}/{} Cost: {:6f}'.format(epoch, n_epoch, loss.item()))

Epoch: 0/10000 Cost: 0.722505
Epoch: 1000/10000 Cost: 0.693147
Epoch: 2000/10000 Cost: 0.693147
Epoch: 3000/10000 Cost: 0.693147
Epoch: 4000/10000 Cost: 0.693147
Epoch: 5000/10000 Cost: 0.693147
Epoch: 6000/10000 Cost: 0.693147
Epoch: 7000/10000 Cost: 0.693147
Epoch: 8000/10000 Cost: 0.693147
Epoch: 9000/10000 Cost: 0.693147


In [6]:
output

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)

# Lab-08-2 Multi Layer Perceptron

## Backpropagation

In [7]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = torch.FloatTensor([[0], [1], [1], [0]])

In [8]:
w1 = torch.rand(2, 2)
b1 = torch.zeros(2)
w2 = torch.rand(2, 1)
b2 = torch.zeros(1)

In [9]:
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

In [10]:
def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [11]:
learning_rate = 1
n_epoch = 10000
for epoch in range(n_epoch):
    # Forward
    z1 = torch.matmul(X, w1) + b1
    a1 = sigmoid(z1)
    z2 = torch.matmul(a1, w2) + b2
    a2 = sigmoid(z2)
    cost = -torch.mean(Y * torch.log(a2) + (1-Y) * torch.log(1-a2))
    
    # Backward
    d_z2 = a2 - Y
    d_w2 = torch.matmul(torch.transpose(a1, 0, 1), d_z2)
    d_b2 = d_z2
    
    d_a1 = torch.matmul(d_z2, torch.transpose(w2, 0, 1))
    d_z1 = d_a1 * sigmoid_prime(z1)
    d_w1 = torch.matmul(torch.transpose(X, 0, 1), d_z1)
    d_b1 = d_z1
    
    # Update weights
    w2 = w2 - learning_rate * d_w2
    b2 = b2 - learning_rate * torch.mean(d_b2, 0)
    w1 = w1 - learning_rate * d_w1
    b1 = b1 - learning_rate * torch.mean(d_b1, 0)
    
    if epoch % 1000 == 0:
        print('Epoch: {}/{} Cost: {:6f}'.format(epoch, n_epoch, cost.item()))

Epoch: 0/10000 Cost: 0.720170
Epoch: 1000/10000 Cost: 0.004116
Epoch: 2000/10000 Cost: 0.001796
Epoch: 3000/10000 Cost: 0.001143
Epoch: 4000/10000 Cost: 0.000836
Epoch: 5000/10000 Cost: 0.000659
Epoch: 6000/10000 Cost: 0.000544
Epoch: 7000/10000 Cost: 0.000462
Epoch: 8000/10000 Cost: 0.000402
Epoch: 9000/10000 Cost: 0.000356


In [12]:
a2

tensor([[6.0415e-04],
        [9.9978e-01],
        [9.9978e-01],
        [2.3343e-04]])

## XOR NN with PyTorch

In [13]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = torch.FloatTensor([[0], [1], [1], [0]])

In [14]:
linear1 = torch.nn.Linear(2, 2, bias=True)
linear2 = torch.nn.Linear(2, 1, bias=True)
sigmoid = torch.nn.Sigmoid()
model = torch.nn.Sequential(linear1,
                            sigmoid,
                            linear2,
                            sigmoid)

In [15]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1)

In [16]:
n_epoch = 10000
for epoch in range(n_epoch):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, Y)
    loss.backward()
    optimizer.step()
    
    if epoch % 1000 == 0:
        print('Epoch: {}/{} Loss: {:6f}'.format(epoch, n_epoch, loss.item()))

Epoch: 0/10000 Loss: 0.699361
Epoch: 1000/10000 Loss: 0.212304
Epoch: 2000/10000 Loss: 0.009619
Epoch: 3000/10000 Loss: 0.004765
Epoch: 4000/10000 Loss: 0.003154
Epoch: 5000/10000 Loss: 0.002354
Epoch: 6000/10000 Loss: 0.001876
Epoch: 7000/10000 Loss: 0.001559
Epoch: 8000/10000 Loss: 0.001333
Epoch: 9000/10000 Loss: 0.001165


In [17]:
output

tensor([[9.8622e-04],
        [9.9908e-01],
        [9.9864e-01],
        [8.6113e-04]], grad_fn=<SigmoidBackward>)

## XOR Deep NN

In [18]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = torch.FloatTensor([[0], [1], [1], [0]])

In [19]:
linear1 = torch.nn.Linear(2, 10, bias=True)
linear2 = torch.nn.Linear(10, 10, bias=True)
linear3 = torch.nn.Linear(10, 10, bias=True)
linear4 = torch.nn.Linear(10, 1, bias=True)
sigmoid = torch.nn.Sigmoid()
model = torch.nn.Sequential(linear1, sigmoid,
                            linear2, sigmoid,
                            linear3, sigmoid,
                            linear4, sigmoid)

In [20]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1)

In [21]:
n_epoch = 10000
for epoch in range(n_epoch):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, Y)
    loss.backward()
    optimizer.step()
    
    if epoch % 1000 == 0:
        print('Epoch: {}/{} Loss: {:6f}'.format(epoch, n_epoch, loss.item()))

Epoch: 0/10000 Loss: 0.817757
Epoch: 1000/10000 Loss: 0.693139
Epoch: 2000/10000 Loss: 0.693130
Epoch: 3000/10000 Loss: 0.693115
Epoch: 4000/10000 Loss: 0.693077
Epoch: 5000/10000 Loss: 0.692920
Epoch: 6000/10000 Loss: 0.686260
Epoch: 7000/10000 Loss: 0.001585
Epoch: 8000/10000 Loss: 0.000553
Epoch: 9000/10000 Loss: 0.000324


In [22]:
output

tensor([[2.0092e-04],
        [9.9977e-01],
        [9.9975e-01],
        [2.2375e-04]], grad_fn=<SigmoidBackward>)